In [1]:
! pip install galois

In [3]:
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit
import numpy as np
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit import ClassicalRegister


In [4]:
def simon_function(s):
    # here we define the input funtion which holds the prommise that 
    # f(x) = f(y) iff y = s^x

    n = len(s)
    qc  = QuantumCircuit(2*n)
    pi = np.random.permutation(2**n) # these are all the permutations of s that are possible.

    # the way we try to build a function that fulfills the promise is with this 
    # function.
    query_gate = np.zeros((4**n, 4**n))

    for x in range(2**n):
        for y in range(2**n):
            z = y^ pi[min(x, x^int(s,2))]
            query_gate[x + 2**n * z, x + 2**n * y] = 1
            
    qc.unitary(query_gate, range(2*n))
    return qc

In [ ]:
 plt.imshow(query_gate)

In [5]:
def simon_measurements ( problem, k):
    #problem is the query function, k is the number of runs
    
    n = problem.num_qubits //2

    qc = QuantumCircuit(2 * n, n)
    qc.h(range(n))
    qc.compose(problem, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))

    result = AerSimulator().run(qc, shots=k, memory=True).result()
    return result.get_memory()
    

In [6]:
simon_measurements(simon_function("11011"),k=12)

['11011',
 '00100',
 '00111',
 '11011',
 '00011',
 '11111',
 '10110',
 '00111',
 '11011',
 '01001',
 '00100',
 '00000']

## This is postprocessing to find the string using the measurements

In [ ]:
import numpy as np
import galois

def simon_algorithm(problem: QuantumCircuit):
    """
    Given a `QuantumCircuit` that implements a query gate for Simon problem, return the hidden string `s`.
    """

    # Quantum part: run the circuit defined previously k times and gather the measurement results.
    # Replace +10 by +r for any nonnegative integer r depending on desired confidence.

    measurements = simon_measurements(problem, k=problem.num_qubits // 2 + 10)
    print("Measurement results:")
    display(measurements)

    # Classical post-processing:

    # 1. Convert measurements of form '11101' to 2D-array of integers
    matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)

    # 2. Interpret matrix as using arithmetic mod 2, and find null space
    null_space = galois.GF(2)(matrix).null_space()
    print("Null space:")
    display(null_space)

    # 3. Convert back to a string
    print("Guess for hidden string s:")
    if len(null_space) == 0:
        # No non-trivial solution; `s` is all-zeros
        return "0" * len(measurements[0])
    return "".join(np.array(null_space[0]).astype(str))